
01 - Data Exploration and Treatment Design
===========================================

Goal: Understand the data structure and identify potential treatment/control splits for DiD.

Key questions to answer:
1. What's the time structure of the data?
2. What are the potential "treatments" we could analyze?
3. What confounders do we need to worry about?
"""

# Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from pathlib import Path
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(project_root)

from src.utils import reconstruct_contact_date


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

pio.templates.default = "plotly_white"


#### Input variables

##### bank client data:
1 - age (numeric)<br>
2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student","blue-collar","self-employed","retired","technician","services") <br>
3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)<br>
4 - education (categorical: "unknown","secondary","primary","tertiary")<br>
5 - default: has credit in default? (binary: "yes","no")<br>
6 - balance: average yearly balance, in euros (numeric) <br>
7 - housing: has housing loan? (binary: "yes","no")<br>
8 - loan: has personal loan? (binary: "yes","no")<br>
##### related with the last contact of the current campaign:
9 - contact: contact communication type (categorical: "unknown","telephone","cellular") <br>
10 - day: last contact day of the month (numeric)<br>
11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")<br>
12 - duration: last contact duration, in seconds (numeric)<br>
##### other attributes:
13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>
14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)<br>
15 - previous: number of contacts performed before this campaign and for this client (numeric)<br>
16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")<br>

##### Output variable (desired target):
17 - y - has the client subscribed a term deposit? (binary: "yes","no")

# Loading

In [2]:
#1) bank-full.csv with all examples, ordered by date (from May 2008 to November 2010).
df=pd.read_csv('data/raw/bank/bank-full.csv',sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
#2)bank-additional-full.csv with all examples, ordered by date (from May 2008 to November 2010).
df2=pd.read_csv('data/raw/bank-additional/bank-additional-full.csv',sep=';')
df2.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# Reconstruct full date from month and day columns
I know from the documentation that data spans May 2008 to November 2010 and is ordered by date. But the year column is missing.


In [4]:
df=reconstruct_contact_date(df,additional_info=False)
df2=reconstruct_contact_date(df2,additional_info=True)

In [5]:
print("Months in data:")
print(df['month'].value_counts().sort_index())

Months in data:
month
apr     2932
aug     6247
dec      214
feb     2649
jan     1403
jul     6895
jun     5341
mar      477
may    13766
nov     3970
oct      738
sep      579
Name: count, dtype: int64


In [6]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,year,contact_date
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,2008,2008-05-05
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,2008,2008-05-05
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,2008,2008-05-05
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,2008,2008-05-05
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,2008,2008-05-05


In [7]:
px.line(
    df['contact_date'].value_counts().sort_index().reset_index(),
    x='contact_date',
    y='count',
    title='Number of Contacts Over Time')

In [8]:
df.loc[(df.contact_date < '2008-10-17')&(df.contact_date > '2008-08-29')]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,year,contact_date


In [9]:
df['campaign'].describe()

count    45211.000000
mean         2.763841
std          3.098021
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         63.000000
Name: campaign, dtype: float64

In [10]:
px.bar(
    df.groupby('campaign', as_index=False)['y'].value_counts(normalize=True),
    x='campaign',
    y='proportion',
    color='y',
    barmode='stack',
    title='Subscription Rate by contact count'
)

In [11]:
#people contacted more than 40 times
df.loc[df.campaign > 40]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,year,contact_date
4275,40,blue-collar,married,secondary,no,-365,yes,no,unknown,19,may,44,43,-1,0,unknown,no,2008,2008-05-19
4299,30,management,single,tertiary,no,358,yes,no,unknown,19,may,88,51,-1,0,unknown,no,2008,2008-05-19
4330,45,management,married,unknown,no,9051,yes,no,unknown,19,may,124,63,-1,0,unknown,no,2008,2008-05-19
4334,42,blue-collar,married,primary,no,170,yes,no,unknown,19,may,51,41,-1,0,unknown,no,2008,2008-05-19
5073,35,technician,married,secondary,no,432,yes,no,unknown,21,may,1094,55,-1,0,unknown,no,2008,2008-05-21
5459,35,blue-collar,married,secondary,no,430,yes,no,unknown,23,may,147,50,-1,0,unknown,no,2008,2008-05-23
10697,28,blue-collar,single,secondary,no,235,yes,no,unknown,16,jun,102,46,-1,0,unknown,no,2008,2008-06-16
11301,28,blue-collar,single,secondary,no,623,no,no,unknown,18,jun,25,41,-1,0,unknown,no,2008,2008-06-18
11914,24,technician,single,primary,no,126,yes,no,unknown,20,jun,10,58,-1,0,unknown,no,2008,2008-06-20
13675,51,blue-collar,married,unknown,no,41,yes,no,telephone,9,jul,16,43,-1,0,unknown,no,2008,2008-07-09


In [12]:
px.histogram(
    df,
    x='campaign',
    title='Distribution of Contact Counts',
    color='y'
)

In [13]:
px.line(
    df2['contact_date'].value_counts().sort_index().reset_index(),
    x='contact_date',
    y='count',
    title='Number of Contacts Over Time [Monthly level]')

# Economic indicator-based treatment

Idea: Economic conditions changed during the campaign.



In [14]:
for m in ['avg_emp_var_rate','avg_cons_price_idx','avg_cons_conf_idx','avg_euribor3m','avg_nr_employed']:
    fig=px.line(
        df2.groupby('contact_date').agg(
            avg_emp_var_rate=('emp.var.rate', 'mean'),
            avg_cons_price_idx=('cons.price.idx', 'mean'),
            avg_cons_conf_idx=('cons.conf.idx', 'mean'),
            avg_euribor3m=('euribor3m', 'mean'),
            avg_nr_employed=('nr.employed', 'mean')
        ).reset_index(),
        x='contact_date',
        y=m,
        title=f'{m} Over Time'
    )
    fig.show()



# Previous contact history
Idea: Customers with no previous contact vs those with previous contact
Then look at current campaign effect


In [15]:
df2.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,year,contact_date,quarter,year_quarter
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2008,2008-05-01,2,2008-Q2
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2008,2008-05-01,2,2008-Q2
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2008,2008-05-01,2,2008-Q2
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2008,2008-05-01,2,2008-Q2
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2008,2008-05-01,2,2008-Q2


In [16]:
df2.loc[df2.pdays == 999,'pdays']=np.nan

In [17]:
df2.columns
for c in ['age', 'job', 'marital', 'education', 'housing', 'loan', 'contact','month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'quarter']:
    print(f"{c}")
    px.histogram(
        df2.groupby([c,'y']).size().reset_index(name='counts'),
        x=c,
        y='counts',
        title=f'Distribution of customer {c} - each contact counted',
        color='y'
    ).show()

age


job


marital


education


housing


...

cons.price.idx


cons.conf.idx


euribor3m


nr.employed


quarter


# Defining wave split

In [18]:
px.line(
    df['contact_date'].value_counts().sort_index().reset_index(),
    x='contact_date',
    y='count',
    title='Number of Contacts Over Time')

In [19]:
# Wave 1: Pre-crisis contacts
wave_1_end = '2008-08-31'
wave_2_start = '2009-04-01'
wave_2_end = '2009-08-01'


df2.loc[(df2['contact_date'] <= pd.to_datetime(wave_1_end)), 'wave'] = 'Wave 1'
df2.loc[(df2['contact_date'] >= pd.to_datetime(wave_2_start))&(df2.contact_date<pd.to_datetime(wave_2_end)), 'wave'] = 'Wave 2'


In [20]:
df2['wave'].value_counts().sort_index().reset_index()

,wave,count
0,Wave 1,23997
1,Wave 2,9145


In [21]:
df.drop_duplicates()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,year,contact_date
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,2008,2008-05-05
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,2008,2008-05-05
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,2008,2008-05-05
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,2008,2008-05-05
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,2008,2008-05-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes,2010,2010-11-17
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes,2010,2010-11-17
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,2010,2010-11-17
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no,2010,2010-11-17


In [22]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,year,contact_date
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,2008,2008-05-05
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,2008,2008-05-05
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,2008,2008-05-05
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,2008,2008-05-05
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,2008,2008-05-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes,2010,2010-11-17
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes,2010,2010-11-17
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,2010,2010-11-17
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no,2010,2010-11-17


In [23]:

# Load both files
bank_add = df2.copy()
bank = df.copy()

print("=== bank-additional.csv ===")
print(f"Total rows: {len(bank_add)}")
print(f"Columns: {bank_add.columns.tolist()}")
print(f"Sample:")
print(bank_add.head(3))

print("\n=== bank.csv ===")
print(f"Total rows: {len(bank)}")
print(f"Columns: {bank.columns.tolist()}")
print(f"Sample:")
print(bank.head(3))

print("\n=== Key differences ===")
print(f"Columns in bank-additional but not in bank: {set(bank_add.columns) - set(bank.columns)}")
print(f"Columns in bank but not in bank-additional: {set(bank.columns) - set(bank_add.columns)}")

=== bank-additional.csv ===
Total rows: 41188
Columns: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'year', 'contact_date', 'quarter', 'year_quarter', 'wave']
Sample:
   age        job  marital    education  default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y  year contact_date  quarter year_quarter    wave
0   56  housemaid  married     basic.4y       no      no   no  telephone   may         mon       261         1    NaN         0  nonexistent           1.1          93.994          -36.4      4.857       5191.0  no  2008   2008-05-01        2      2008-Q2  Wave 1
1   57   services  married  high.school  unknown      no   no  telephone   may         mon       149       

In [24]:
# Check if there's any way to identify unique customers
print("\n=== Checking for customer identification ===")

# Method 1: Is there an explicit ID?
if 'customer_id' in bank_add.columns or 'id' in bank_add.columns:
    print("✅ Customer ID found!")
else:
    print("❌ No explicit customer ID")

# Method 2: Can we create a pseudo-ID from demographics?
# (Age + Job + Marital + Education might uniquely identify customers)
pseudo_id_cols = ['age', 'job', 'marital', 'education','housing','loan','contact']
bank_add['pseudo_id'] = bank_add[pseudo_id_cols].astype(str).agg('_'.join, axis=1)

print(f"\nTotal contact rows: {len(bank_add)}")
print(f"Unique pseudo-IDs: {bank_add['pseudo_id'].nunique()}")
print(f"Ratio: {bank_add['pseudo_id'].nunique() / len(bank_add):.2%}")

# If ratio is close to 100%, pseudo-ID might work
# If ratio is low (e.g., 50%), many customers contacted multiple times


=== Checking for customer identification ===
❌ No explicit customer ID

Total contact rows: 41188
Unique pseudo-IDs: 14010
Ratio: 34.01%


In [25]:
# How many customers were contacted in BOTH waves?

# Define waves
bank_add['wave'] = 'other'
bank_add.loc[bank_add['contact_date'] <= '2008-08-31', 'wave'] = 'wave_1'
bank_add.loc[(bank_add['contact_date'] >= '2009-04-01') & 
             (bank_add['contact_date'] < '2009-08-01'), 'wave'] = 'wave_2'

# Group by pseudo-ID
customer_wave_counts = bank_add[bank_add['wave'].isin(['wave_1', 'wave_2'])].groupby('pseudo_id')['wave'].nunique()

print(f"\nCustomers contacted in only Wave 1 or Wave 2: {(customer_wave_counts == 1).sum()}")
print(f"Customers contacted in BOTH waves: {(customer_wave_counts == 2).sum()}")

if (customer_wave_counts == 2).sum() > 0:
    print("⚠️ WARNING: Cross-wave contamination detected!")
    print("These customers can't be cleanly assigned to treatment or control")


Customers contacted in only Wave 1 or Wave 2: 9858
Customers contacted in BOTH waves: 2112
⚠️ WARNING: Cross-wave contamination detected!
These customers can't be cleanly assigned to treatment or control


# Creating clean dataset

In [26]:
# %% Create pseudo-ID
pseudo_id_cols = ['age', 'job', 'marital', 'education', 'housing', 'loan', 'contact']
df2['pseudo_id'] = df2[pseudo_id_cols].astype(str).agg('_'.join, axis=1)

# %% Define waves
df2['wave'] = 'other'
df2.loc[df2['contact_date'] <= '2008-08-31', 'wave'] = 'wave_1'
df2.loc[(df2['contact_date'] >= '2009-04-01') & 
       (df2['contact_date'] < '2009-08-01'), 'wave'] = 'wave_2'

# %% Identify single-wave customers
customer_waves = df2[df2['wave'].isin(['wave_1', 'wave_2'])].groupby('pseudo_id')['wave'].nunique()
single_wave_customers = customer_waves[customer_waves == 1].index

print(f"Single-wave customers: {len(single_wave_customers)}")
print(f"Cross-wave customers (excluded): {(customer_waves > 1).sum()}")

# %% Create clean analysis dataset
df_clean = df2[df2['pseudo_id'].isin(single_wave_customers) & 
              df2['wave'].isin(['wave_1', 'wave_2'])].copy()

print(f"\nClean dataset:")
print(f"Total contacts: {len(df_clean)}")
print(f"Unique customers: {df_clean['pseudo_id'].nunique()}")
print(f"Wave 1 contacts: {(df_clean['wave']=='wave_1').sum()}")
print(f"Wave 2 contacts: {(df_clean['wave']=='wave_2').sum()}")
print(f"Avg contacts per customer: {len(df_clean) / df_clean['pseudo_id'].nunique():.1f}")

Single-wave customers: 9858
Cross-wave customers (excluded): 2112

Clean dataset:
Total contacts: 18291
Unique customers: 9858
Wave 1 contacts: 15001
Wave 2 contacts: 3290
Avg contacts per customer: 1.9


In [27]:
# %% Covariate balance on clean sample
covariates = ['age', 'campaign', 'pdays', 'previous', 
              'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

# Calculate at CONTACT level (since we have ~3 contacts per customer)
balance = df_clean.groupby('wave')[covariates].mean().T
balance.columns = ['Wave 1', 'Wave 2']
balance['Difference'] = balance['Wave 1'] - balance['Wave 2']
balance['Abs % Diff'] = (abs(balance['Difference']) / balance['Wave 1'] * 100).round(1)

print("\n=== COVARIATE BALANCE TABLE ===")
print(balance)

# %% Highlight key findings
print("\n=== ECONOMIC INDICATORS (should differ) ===")
econ_vars = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
print(balance.loc[econ_vars, ['Wave 1', 'Wave 2', 'Difference']])

print("\n=== CUSTOMER CHARACTERISTICS (should be similar) ===")
customer_vars = ['age', 'campaign', 'pdays', 'previous']
print(balance.loc[customer_vars, ['Wave 1', 'Wave 2', 'Abs % Diff']])


=== COVARIATE BALANCE TABLE ===
                     Wave 1       Wave 2  Difference  Abs % Diff
age               41.371442    39.598784    1.772658         4.3
campaign           2.938671     2.238906    0.699765        23.8
pdays                   NaN     5.804878         NaN         NaN
previous           0.000000     0.321884   -0.321884         inf
emp.var.rate       1.269669    -1.916687    3.186356       251.0
cons.price.idx    94.029737    92.942840    1.086897         1.2
cons.conf.idx    -38.993747   -45.737143    6.743396       -17.3
euribor3m          4.909584     1.318218    3.591366        73.2
nr.employed     5211.982361  5096.670790  115.311571         2.2

=== ECONOMIC INDICATORS (should differ) ===
                     Wave 1       Wave 2  Difference
emp.var.rate       1.269669    -1.916687    3.186356
cons.price.idx    94.029737    92.942840    1.086897
cons.conf.idx    -38.993747   -45.737143    6.743396
euribor3m          4.909584     1.318218    3.591366
nr.empl

In [28]:
# %% Are cross-wave customers different from single-wave customers?
df2['is_cross_wave'] = df2['pseudo_id'].isin(customer_waves[customer_waves > 1].index)

comparison = df2[df2['wave'].isin(['wave_1', 'wave_2'])].groupby('is_cross_wave')[
    ['age', 'campaign', 'previous', 'emp.var.rate']
].mean()

comparison.index = ['Single-wave customers', 'Cross-wave customers (EXCLUDED)']
print("\n=== CROSS-WAVE VS SINGLE-WAVE COMPARISON ===")
print(comparison)

# Interpretation guide
print("\nIf cross-wave customers have:")
print("- Higher 'previous': They were contacted in earlier campaigns (selection bias)")
print("- Higher 'campaign': They received more intensive contact (selection bias)")
print("- Different age: Bank targeted different demographics over time")


=== CROSS-WAVE VS SINGLE-WAVE COMPARISON ===
                                       age  campaign  previous  emp.var.rate
Single-wave customers            41.052594  2.812804  0.057897      0.696539
Cross-wave customers (EXCLUDED)  37.673490  2.636927  0.128409      0.072938

If cross-wave customers have:
- Higher 'previous': They were contacted in earlier campaigns (selection bias)
- Higher 'campaign': They received more intensive contact (selection bias)
- Different age: Bank targeted different demographics over time
